### This notebook is used for quick linear fitting

For a general case using features: $\{x_1, x_2, ... x_m\}$ to predict target y

1) y might depends on $x_i '$ to the power of p, so each feature $x_i$ will be expand to a set of new features including $x_i$ to the power of [$1 \over 2$, ${1}\over{3}$, $1\over 4$,1,2, 3, 4]

2) greedy choose features that predict target the best

3) add interation terms between chosen features and do greedy search again

3) fit the model with chosen features

In [7]:
import pandas as pd
import numpy as np
from numpy import meshgrid
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

In [8]:
# for testing purpose, use genXY to generate test sets
def genXY(d_row, d_feature):
    X = np.random.randn(d_row, d_feature)
    y = X.dot(np.random.randint(-5, 5, d_feature)) + 2*np.random.randn(d_row)
    return X, y

In [9]:
def loadDF(path):
    df = pd.read_csv(path)
    corr = df.corr()
    corr.style.background_gradient()
    #print(corr)
    display(corr)
    return df

In [11]:
path = "PredictiveModelingAssessmentData.csv"

In [12]:
df = pd.read_csv(path)

In [13]:
# correlation heat map
corr = df.corr()
corr.style.background_gradient().set_precision(1)

,y,x1,x2
y,1,0.3,0.6
x1,0.3,1,0.007
x2,0.6,0.007,1


In [14]:
# fn corrMap to quickly print corralation heat map for a df
def corrMap(df):
    corr = df.corr()
    corr.style.background_gradient()
    display(corr)

In [15]:
# expand current features to pows
def expandFeatures0(df, cols, pows):
    res_df = df.copy()
    for col in cols:
        for p in pows:
            new_col = str(p) + "_" + col
            res_df[new_col] = np.power(res_df[col], p)
    return res_df

def expandFeatures1(df, cols_pows):
    res_df = df.copy()
    for col, pows in cols_pows.items():
        for p in pows:
            new_col = str(np.round(p, 2)) + '_' + col
            res_df[new_col] = np.power(res_df[col], p)
    return res_df


In [16]:
cols = list(df.columns)
target = 'y'
cols.remove(target)
pows = [1/2, np.round(1/3, 2), 2, 4]
# make a dic for desired expansion for each features
cols_pows = {"x1": [2,3,4], "x2": [1/2,np.round(1/3, 2), 2, 3, 4]}
print("the columns to be expanded: ", cols)
print("the powers to be added as new features: ", pows)

the columns to be expanded:  ['x1', 'x2']
the powers to be added as new features:  [0.5, 0.33, 2, 4]


In [17]:
test_df = expandFeatures1(df, cols_pows)
test_df.head()

,y,x1,x2,2_x1,3_x1,4_x1,0.5_x2,0.33_x2,2_x2,3_x2,4_x2
0,1.300215,-0.054425,0.738897,0.002962,-0.000161,0.000009,0.859591,0.904967,0.545968,0.403414,0.298081
1,-0.805025,0.130174,0.977855,0.016945,0.002206,0.000287,0.988866,0.992637,0.956201,0.935026,0.914320
2,2.801926,1.749007,1.352562,3.059024,5.350253,9.357628,1.162997,1.104795,1.829423,2.474407,3.346788
3,3.123490,-0.979458,1.664484,0.959338,-0.939632,0.920330,1.290149,1.183102,2.770506,4.611463,7.675705
4,3.445728,0.300521,2.988848,0.090313,0.027141,0.008156,1.728829,1.435213,8.933214,26.700023,79.802321


In [18]:
# greedy features selection
def loss(model, X_test, y_test):
    pred = model.predict(X_test)
    return np.sum(np.power(pred-y_test, 2))/len(y_test)

def get_X_y(df, features, target):
    return np.array(df[features]), np.array(df[target])

def build_one(df_train, df_test, feature, target):
    reg_temp = LinearRegression(copy_X = True)
    X_train, y_train = get_X_y(df_train, feature, target)
    X_test, y_test = get_X_y(df_test, feature, target)
    reg_temp.fit(X_train, y_train)
    loss_temp = loss(reg_temp, X_test, y_test)
    return reg_temp, loss_temp, feature

def build_first(df_train, df_test, features, target):
    reg = LinearRegression(copy_X = True)
    cur_model, cur_loss, cur_feature = build_one(df_train, df_test, [features[0]], target)
    for f in features:
        new_model, new_loss, new_feature = build_one(df_train, df_test, [f], target)
        if new_loss < cur_loss:
            cur_model, cur_loss, cur_feature = new_model, new_loss, new_feature
    return cur_model, cur_loss, cur_feature

def build_new(cur_model, cur_loss, df_train, df_test, cur_features, new_features, target, flag):
    previous_loss = cur_loss
    
    for new_f in new_features:
        temp_features = cur_features + [new_f]
        reg_temp = LinearRegression(copy_X = True)
        X_train, y_train = get_X_y(df_train, temp_features, target)
        X_test, y_test = get_X_y(df_test, temp_features, target)
        reg_temp.fit(X_train, y_train)
        loss_temp = loss(reg_temp, X_test, y_test)
        if loss_temp < cur_loss:
            cur_model, cur_loss, cur_features = reg_temp, loss_temp, temp_features
    if cur_loss == previous_loss:
        flag = False
    return cur_model, cur_loss, cur_features, flag
        
def greedyForwards(df, features,target):
    # function loss(model, X_test, y_test) to compute the loss function of model 
    # first build new Model for each feature and select the best one -- build_one(features)
    # function build new Model build_new(cur_features, features_to_add, cur_model)
    # for each feature_to_add f, add f to cur_features, and build a ne multi linear regression model new_model, compare its loss with 
    # the current model by comparing loss(new_model, test_X, test_y) and loss(model, test_X, test_y) 
    # continue if the new model is better else stop and use the current model
    
    # first split the data into train and test sets
    df_copy = df.copy()
    df_train = df_copy.sample(frac = 0.75, random_state=404)
    #print("length,",len(df_train))
    df_test = df_copy.drop(df_train.index)
    # build first model with one feature
    model0, loss0, f0 = build_first(df_train, df_test, features, target)
    #build_new(cur_model, cur_loss, df_train, df_test, cur_features, new_features target, flag)
    flag = True
    model, loss, fs, flag = model0, loss0, f0, flag
    while flag:
        new_features = list(set(features)-set(fs))
        model, loss, fs, flag = build_new(model, loss, df_train, df_test, fs, new_features, target, flag)
    return model, fs, loss

In [19]:
test_fs = list(test_df.columns)
test_fs.remove(target)
print(test_fs, target)

['x1', 'x2', '2_x1', '3_x1', '4_x1', '0.5_x2', '0.33_x2', '2_x2', '3_x2', '4_x2'] y


In [20]:
test_df.head()

,y,x1,x2,2_x1,3_x1,4_x1,0.5_x2,0.33_x2,2_x2,3_x2,4_x2
0,1.300215,-0.054425,0.738897,0.002962,-0.000161,0.000009,0.859591,0.904967,0.545968,0.403414,0.298081
1,-0.805025,0.130174,0.977855,0.016945,0.002206,0.000287,0.988866,0.992637,0.956201,0.935026,0.914320
2,2.801926,1.749007,1.352562,3.059024,5.350253,9.357628,1.162997,1.104795,1.829423,2.474407,3.346788
3,3.123490,-0.979458,1.664484,0.959338,-0.939632,0.920330,1.290149,1.183102,2.770506,4.611463,7.675705
4,3.445728,0.300521,2.988848,0.090313,0.027141,0.008156,1.728829,1.435213,8.933214,26.700023,79.802321


In [21]:
model0, selected_features, loss0 = greedyForwards(test_df, test_fs, target)
print("selected_features are: ", selected_features, "the cost of current model is: ", np.round(loss0, 4))

selected_features are:  ['0.33_x2', 'x2', '2_x2', '0.5_x2', 'x1', '2_x1', '4_x2', '4_x1'] the cost of current model is:  1.2529


In [22]:
#corrMap(test_df)
test_corr = test_df.corr()
test_corr.style.background_gradient()

,y,x1,x2,2_x1,3_x1,4_x1,0.5_x2,0.33_x2,2_x2,3_x2,4_x2
y,1,0.262895,0.631638,0.288778,0.2175,0.187632,0.690903,0.705617,0.492209,0.370317,0.281204
x1,0.262895,1,0.00658,-0.0143724,0.786745,-0.0239852,0.005115,0.00488736,0.0102168,0.0125753,0.0133092
x2,0.631638,0.00658,1,-0.00848253,0.00873011,-0.0199498,0.973844,0.950817,0.929242,0.787972,0.647022
2_x1,0.288778,-0.0143724,-0.00848253,1,-0.0384283,0.887963,-0.0078007,-0.00751554,-0.00850772,-0.00702759,-0.00562766
3_x1,0.2175,0.786745,0.00873011,-0.0384283,1,-0.077136,0.0107448,0.0117543,0.00725429,0.00721513,0.00726032
4_x1,0.187632,-0.0239852,-0.0199498,0.887963,-0.077136,1,-0.0209162,-0.0211151,-0.0175271,-0.0150875,-0.0130758
0.5_x2,0.690903,0.005115,0.973844,-0.0078007,0.0107448,-0.0209162,1,0.996126,0.830524,0.659352,0.514722
0.33_x2,0.705617,0.00488736,0.950817,-0.00751554,0.0117543,-0.0211151,0.996126,1,0.785933,0.609579,0.467565
2_x2,0.492209,0.0102168,0.929242,-0.00850772,0.00725429,-0.0175271,0.830524,0.785933,1,0.953339,0.857827
3_x2,0.370317,0.0125753,0.787972,-0.00702759,0.00721513,-0.0150875,0.659352,0.609579,0.953339,1,0.970374


In [23]:
# Now we have the primary selected features, interation terms are to be added. The strategy of choosing interactions is subjective
# Before a better method is found, get interactions for all combinations
# fn addInteraction:
def addInteractions(df, selected_features):
    res_df = df.copy()
    for i in range(len(selected_features)):
        for j in range(i+1, len(selected_features)):
            f1, f2 = selected_features[i], selected_features[j]
            new_col = 'cross_' + f1 + "_" + f2
            res_df[new_col] = res_df[f1] * res_df[f2]
    return res_df

In [24]:
cross_df = addInteractions(test_df, selected_features)
print(list(cross_df.columns))

['y', 'x1', 'x2', '2_x1', '3_x1', '4_x1', '0.5_x2', '0.33_x2', '2_x2', '3_x2', '4_x2', 'cross_0.33_x2_x2', 'cross_0.33_x2_2_x2', 'cross_0.33_x2_0.5_x2', 'cross_0.33_x2_x1', 'cross_0.33_x2_2_x1', 'cross_0.33_x2_4_x2', 'cross_0.33_x2_4_x1', 'cross_x2_2_x2', 'cross_x2_0.5_x2', 'cross_x2_x1', 'cross_x2_2_x1', 'cross_x2_4_x2', 'cross_x2_4_x1', 'cross_2_x2_0.5_x2', 'cross_2_x2_x1', 'cross_2_x2_2_x1', 'cross_2_x2_4_x2', 'cross_2_x2_4_x1', 'cross_0.5_x2_x1', 'cross_0.5_x2_2_x1', 'cross_0.5_x2_4_x2', 'cross_0.5_x2_4_x1', 'cross_x1_2_x1', 'cross_x1_4_x2', 'cross_x1_4_x1', 'cross_2_x1_4_x2', 'cross_2_x1_4_x1', 'cross_4_x2_4_x1']


In [25]:
cross_fs = list(cross_df.columns)
cross_fs.remove(target)
model1, cross_selected_features, loss1 = greedyForwards(cross_df, cross_fs, target)

In [26]:
print("selected_features are: ", cross_selected_features, "the cost of current model is: ", np.round(loss1, 4))

selected_features are:  ['0.33_x2', 'cross_x2_4_x1', 'cross_2_x2_0.5_x2', 'cross_x1_4_x2', 'cross_x2_x1', 'cross_0.33_x2_x1', 'cross_0.33_x2_x2', 'cross_x1_4_x1', 'x1', 'cross_2_x2_x1', 'cross_0.33_x2_2_x1', '2_x2', 'cross_0.5_x2_x1', 'cross_2_x1_4_x2', 'cross_x2_2_x1', '3_x1', 'cross_2_x2_4_x2', 'cross_0.5_x2_2_x1', '2_x1', 'cross_2_x2_2_x1', 'cross_x1_2_x1'] the cost of current model is:  1.0298


In [27]:
cross_corr = cross_df[[target]+cross_selected_features].corr()
cross_corr.style.background_gradient()

,y,0.33_x2,cross_x2_4_x1,cross_2_x2_0.5_x2,cross_x1_4_x2,cross_x2_x1,cross_0.33_x2_x1,cross_0.33_x2_x2,cross_x1_4_x1,x1,cross_2_x2_x1,cross_0.33_x2_2_x1,2_x2,cross_0.5_x2_x1,cross_2_x1_4_x2,cross_x2_2_x1,3_x1,cross_2_x2_4_x2,cross_0.5_x2_2_x1,2_x1,cross_2_x2_2_x1,cross_x1_2_x1
y,1,0.705617,0.399675,0.427389,0.0960533,0.21899,0.254183,0.586157,0.162435,0.262895,0.160798,0.452797,0.492209,0.246802,0.329883,0.583074,0.2175,0.178301,0.508837,0.288778,0.527143,0.2175
0.33_x2,0.705617,1,0.179809,0.695023,0.0375877,0.0115926,0.00530057,0.903,0.0184699,0.00488736,0.0236679,0.160428,0.785933,0.00644927,0.329219,0.367965,0.0117543,0.293067,0.230986,-0.00751554,0.429262,0.0117543
cross_x2_4_x1,0.399675,0.179809,1,0.144782,0.0195283,0.00608226,0.00426225,0.178531,0.00136511,0.00186082,0.00811845,0.848586,0.161173,0.00505802,0.408413,0.883178,0.0043649,0.052932,0.877278,0.738562,0.731365,0.0043649
cross_2_x2_0.5_x2,0.427389,0.695023,0.144782,1,0.10789,0.0371785,0.0182375,0.919139,0.00524542,0.0116082,0.069616,0.10688,0.98676,0.0224495,0.628597,0.327962,0.00716143,0.761475,0.167409,-0.00785326,0.531469,0.00716143
cross_x1_4_x2,0.0960533,0.0375877,0.0195283,0.10789,1,0.564136,0.336682,0.0712444,0.0886912,0.231772,0.836294,0.0188881,0.093666,0.394028,0.196574,0.0508651,0.15615,0.1274,0.0261648,0.00718218,0.105664,0.15615
cross_x2_x1,0.21899,0.0115926,0.00608226,0.0371785,0.564136,1,0.921805,0.0241765,0.40154,0.813393,0.894794,0.00299735,0.0322812,0.958109,0.0605406,0.01281,0.617124,0.0403072,0.00521825,-0.000633998,0.0308892,0.617124
cross_0.33_x2_x1,0.254183,0.00530057,0.00426225,0.0182375,0.336682,0.921805,1,0.0117203,0.508143,0.97107,0.682645,-0.00252355,0.0158268,0.993804,0.0283353,0.00478093,0.753746,0.0192787,-0.000439911,-0.00719828,0.0137972,0.753746
cross_0.33_x2_x2,0.586157,0.903,0.178531,0.919139,0.0712444,0.0241765,0.0117203,1,0.0100948,0.00782274,0.0460431,0.141379,0.969778,0.014425,0.505044,0.379753,0.00796365,0.520565,0.213164,-0.00872217,0.52698,0.00796365
cross_x1_4_x1,0.162435,0.0184699,0.00136511,0.00524542,0.0886912,0.40154,0.508143,0.0100948,1,0.54312,0.248511,-0.0285252,0.00674095,0.483554,0.00489316,0.00674066,0.914807,0.0029303,-0.014726,-0.0670813,0.0096318,0.914807
x1,0.262895,0.00488736,0.00186082,0.0116082,0.231772,0.813393,0.97107,0.00782274,0.54312,1,0.540806,-0.00641108,0.0102168,0.939517,0.0170453,0.0019988,0.786745,0.0118053,-0.00355027,-0.0143724,0.00828679,0.786745


In [30]:
cross_corr['y'].sort_values(ascending = False)

y                     1.000000
0.33_x2               0.705617
cross_0.33_x2_x2      0.586157
cross_x2_2_x1         0.583074
cross_2_x2_2_x1       0.527143
cross_0.5_x2_2_x1     0.508837
2_x2                  0.492209
cross_0.33_x2_2_x1    0.452797
cross_2_x2_0.5_x2     0.427389
cross_x2_4_x1         0.399675
cross_2_x1_4_x2       0.329883
2_x1                  0.288778
x1                    0.262895
cross_0.33_x2_x1      0.254183
cross_0.5_x2_x1       0.246802
cross_x2_x1           0.218990
cross_x1_2_x1         0.217500
3_x1                  0.217500
cross_2_x2_4_x2       0.178301
cross_x1_4_x1         0.162435
cross_2_x2_x1         0.160798
cross_x1_4_x2         0.096053
Name: y, dtype: float64